# kachaka_api 函式庫

- kachaka_api 函式庫可以讓您在不需要意識到 gRPC 通訊的情況下使用 Kachaka 的 API。

## 關於非同步函式庫
- 在此範例中，我們使用 kachaka_api.aio.KachakaApiClient 類別，以方法呼叫的形式執行 Kachaka 的 API。
- 同步函式庫也已準備好。詳情請參閱[同步函式庫篇](./kachaka_api_client.ipynb)。

#### 安裝相依函式庫
- 預設情況下不會安裝 matplotlib 和 numpy。請根據需要進行安裝。
- 執行 pip install 後，請重新啟動 JupyterLab 的核心（kernel）

In [ ]:
%pip install -q matplotlib numpy
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, clear_output, display

## 匯入與建立客戶端
- 建立客戶端。只需要執行一次即可。
- 透過在引數中指定 `<Kachaka的IP位址:26400>`，可以在外部的電腦等裝置上使用本函式庫

In [ ]:
import kachaka_api

client = kachaka_api.aio.KachakaApiClient()

## 各 API 的執行

- 以下提供各 API 的執行範例。

### 取得機器人的個體資訊

#### 取得機器人的序號

In [ ]:
await client.get_robot_serial_number()

#### 取得 Kachaka 的軟體版本資訊

In [ ]:
await client.get_robot_version()

### 取得家具・目的地的資訊

#### 取得目的地資訊列表
- id 用於後述的函式中指定目的地時使用
- pose 的單位與機器人姿態相同，為 m 和 radian

In [ ]:
await client.get_locations()

#### 取得預設目的地的 ID
- 預設目的地是在省略目的地時（例如「把○○帶過來」）所使用的目的地

In [ ]:
await client.get_default_location_id()

#### 取得家具資訊列表
- id 用於後述的函式中指定家具時使用
- pose 的單位與機器人姿態相同，為 m 和 radian

In [ ]:
await client.get_shelves()

#### 取得對接中的家具 ID
- id 會是上面取得的列表中的某一個，或者在未對接時返回空字串。

In [ ]:
await client.get_moving_shelf_id()

### 執行指令

#### 移動家具
- 指定家具 ID 和目的地 ID 後，會將指定的家具搬運到目的地。
- 各 ID 的列表可分別透過前述的 get_shelves()、get_locations() 取得。

In [ ]:
await client.move_shelf("S01", "L01")

#### 以名稱指定家具和目的地
* kachaka_api 套件也可以透過名稱來指定家具和目的地。
* 如下所示，呼叫 `update_resolver` 即可更新名稱對照表。
* 當家具或目的地有增減時，需要再次呼叫 `update_resolver` 以反映變更。

In [ ]:
await client.update_resolver()
await client.move_shelf("シェルフ", "ダイニング")

#### 收回家具
- 收回指定的家具。

In [ ]:
await client.return_shelf("S01")

- 省略參數時，會收回目前承載的家具。

In [ ]:
await client.return_shelf()

#### 放下家具
- 將承載的家具放置在原地。

In [ ]:
await client.undock_shelf()

#### 移動到目的地

In [ ]:
await client.move_to_location("L01")

#### 前往充電座

In [ ]:
await client.return_home()

#### 承載家具
- 承載 Kachaka 前方的家具

In [ ]:
await client.dock_shelf()

#### 不論哪個家具，承載指定目的地的家具
- 前往指定的目的地，不論是哪個家具，承載放置在該處的家具。
- 如果是未註冊的家具，會作為新家具進行註冊。
- 第 1 個參數指定目的地的 ID，第 2 個參數指定對接方向（頭部朝前: True，尾部朝前: False）。

In [ ]:
await client.dock_any_shelf_with_registration("L01", False)

#### 語音發話

In [ ]:
await client.speak("こんにちは、カチャカです")

#### 指定地圖上座標的移動

In [ ]:
await client.move_to_pose(0.5, 0.0, 0)

#### 前後方向的移動
* 以公尺指定距離後，會前進該距離。（負值表示後退）
* （可選）可在 speed 中指定速度 [m/s]。（準確地說，是以此為最高速度進行回饋控制）

In [ ]:
await client.move_forward(0.5)
await client.move_forward(-0.4, speed=0.3)

#### 原地旋轉
* 以弧度指定角度後，會以逆時針方向（繞 Z 軸右手定則方向）原地旋轉。

In [ ]:
import math

await client.rotate_in_place(math.pi)

#### 自我位置修正
* 當 Kachaka 在地圖上的位置與實際位置有偏差時，Kachaka 會旋轉觀察周圍環境，嘗試修正位置偏差。
* 如果修正失敗，Kachaka 的位置不會被修正。

In [ ]:
await client.localize()

### 指令的執行狀態與管理

#### 取消正在執行的指令

In [ ]:
await client.cancel_command()

#### 取得正在執行的指令狀態
- 查詢指令是否正在執行中

In [ ]:
await client.is_command_running()

- 取得正在執行的指令

In [ ]:
await client.get_running_command()

#### 取得最後執行的指令結果

In [ ]:
await client.get_last_command_result()

#### 取得指令執行歷史

In [ ]:
await client.get_history_list()

#### 待機狀態及其解除

執行 lock() 可以讓 Kachaka 保持在原地不做任何動作的狀態。
與在 Python 程式中使用 sleep 不同，Kachaka 會處於執行「待機」指令的狀態。接著以 cancel_all=False 執行指令時，待機狀態解除後會進入下一個指令。

In [ ]:
await client.lock(30.0)
await client.speak("待機状態が解除されました", cancel_all=False)

當時間經過或執行 proceed() 後，待機狀態就會結束。

In [ ]:
await client.proceed()

### 捷徑
可以取得和執行智慧型手機應用程式中的捷徑列表。

#### 取得列表
可以以 dict 形式取得 ID 和捷徑名稱的配對。

In [ ]:
await client.get_shortcuts()

#### 執行捷徑

In [ ]:
shortcut_id = min((await client.get_shortcuts()).keys())
await client.start_shortcut_command(shortcut_id, True)

#### 重設家具位置
* 當非 Kachaka 對接狀態下有人手動移動家具位置時，Kachaka 的認知與實際位置可能會產生偏差。
* 將家具放回主位置後呼叫此方法，即可校正認知。

In [ ]:
await client.reset_shelf_pose("S01")

#### 切換自動充電的開啟/關閉
- 當一段時間未接收到速度指令時，Kachaka 會自動返回充電站。這稱為自動充電。

In [ ]:
await client.set_auto_homing_enabled(True)

### 音量設定
可以將音量設定為 0-10。

#### 取得目前的音量

In [ ]:
await client.get_speaker_volume()

#### 設定音量

In [ ]:
await client.set_speaker_volume(10)

### 速度指令

#### 切換為手動操作模式
- 要透過速度指令移動 Kachaka，需要切換到手動操控模式。
- 60 秒後會自動關閉，因此需要根據需要定期重新啟用。
- 在充電座上時，會先前進離開充電座

In [ ]:
await client.set_manual_control_enabled(True)

#### 確認手動操控模式

In [ ]:
await client.get_manual_control_enabled()

#### 速度指令
- 指令的值會保持到下一個指令值到來，或經過 0.3 秒為止
- 單位為 m/s 和 rad/s

In [ ]:
for i in range(100):
    await client.set_robot_velocity(0.0, 0.3)

### 取得各種感測器資料等

#### 取得充電狀態・電量
* 返回 (電量(%), 充電狀態)。
* 充電狀態為以下之一。proto 中定義了其他值，但未被使用。
  * 充電中: `PowerSupplyStatus.POWER_SUPPLY_STATUS_CHARGING` (=1)
  * 放電中: `PowerSupplyStatus.POWER_SUPPLY_STATUS_DISCHARGING` (=2)

In [ ]:
await client.get_battery_info()

#### 取得地圖上的姿態
- 單位為 m 和 radian

In [ ]:
await client.get_robot_pose()

#### 取得地圖資訊
- 有關地圖圖片與地圖座標的對應關係，請參閱 [plot_map_robot_lidar.ipynb](../python/demos/plot_map_robot_lidar.ipynb)

In [ ]:
map = await client.get_png_map()
print(map.name)
print(map.resolution, map.width, map.height)
print(map.origin)
display(Image(data=map.data))

#### 取得 LiDAR 點雲資訊
- LiDAR 在充電座上時會停止運作，資料不會更新

In [ ]:
scan = await client.get_ros_laser_scan()

theta = np.linspace(scan.angle_min, scan.angle_max, len(scan.ranges))
dist = np.array(scan.ranges)

# 顯示 LiDAR 點雲的範例
plt.scatter(dist * np.cos(theta), dist * np.sin(theta))

#### 取得 IMU 資訊

In [ ]:
await client.get_ros_imu()

#### 取得里程計資料

In [ ]:
await client.get_ros_odometry()

### 相機相關

#### 取得相機資訊

In [ ]:
await client.get_front_camera_ros_camera_info()

#### 取得前置相機影像
##### 使用影像串流
- 透過使用 aio 的 stream API，可以避免重複取得相同影像等問題（請參閱 gRPC 中 cursor 的概念）
- 雖然也有 `get_front_camera_ros_compressed_image()` 函式，但這是不使用 cursor 的簡易程式碼，連續呼叫時可能會取得兩張相同的影像
- 此外，如果需要無損資料，可以使用 `front_camera_ros_image` 取得。但請注意資料量會增加。

In [ ]:
image_stream = client.front_camera_ros_compressed_image.stream()

##### 取得單張影像的方法
- 從串流中取得最新的一張影像。

In [ ]:
image = await image_stream.__anext__()
display(Image(data=image.data, format="jpeg"))

##### 連續取得影像的方法
- 可以使用 JupyterLab 的 ■ 按鈕停止

In [ ]:
async for image in image_stream:
    clear_output(wait=True)
    display(Image(data=image.data, format="jpeg"))

#### 取得物體偵測結果
- 取得相機所看到的物體（充電器、家具、人）的資訊。
- 需要在啟動相機後才能呼叫。
- 繪製到影像上的方法請參閱[物體偵測範例](../python/demos/get_object_detection.ipynb)

In [ ]:
await client.get_object_detection()

### 地圖的匯入・匯出

#### 取得地圖列表

In [ ]:
map_list = await client.get_map_list()
for map_list_entry in map_list:
    print("id:", map_list_entry.id)
    print("name:", map_list_entry.name)

#### 取得目前使用中的地圖 ID

In [ ]:
current_map_id = await client.get_current_map_id()
current_map_id

#### 取得地圖預覽
* 可以從上面取得的地圖列表中，指定 ID 來取得該地圖的預覽（PNG 格式的地圖影像）。
* 這裡展示指定目前使用中的地圖 ID 的範例。（如果是目前使用中的地圖，也可以用 client.get_png_map() 取得）

In [ ]:
current_map_preview = await client.load_map_preview(current_map_id)
display(Image(data=current_map_preview.data, format="png"))

#### 匯出地圖
* 透過匯出地圖，可以進行備份或在其他 Kachaka 機體上重複使用。
* 可以從上面取得的地圖列表中，指定 ID 進行匯出。
* 會以二進位檔案的形式儲存到第 2 個參數指定的路徑。
* 資料為專有格式的二進位。僅能透過匯入 API 使用。

In [ ]:
EXPORT_TARGET_FILE_PATH = "current_map"
await client.export_map(current_map_id, "current_map")

#### 匯入地圖
* 可以將上面匯出 API 輸出的檔案匯入到其他機體等。

In [ ]:
await client.import_map(EXPORT_TARGET_FILE_PATH)

### 啟動狀態管理

#### 重新啟動 Kachaka
從執行到重新啟動開始、LED 開始旋轉閃爍，可能需要數秒鐘。

In [ ]:
await client.restart_robot()

### 暫停

#### 以軟體方式將暫停按鈕設為按下狀態

In [ ]:
await client.set_emergency_stop()